Mô phỏng quản lý bãi đỗ xe máy dùng danh sách liên kết. Thông tin các xe vào gửi gồm 
+ biển số xe
+ thời gian gửi
+ ảnh xe khi vào (nếu không có thì tạm bỏ qua)
+ mã thẻ gửi xe
Khi lấy ra sẽ có 
+ mã thẻ gửi
+ biển số xe lấy ra
+ ảnh xe khi lấy (nếu không có thfi bỏ qua)
+ thời điểm lấy
Kiểm tra xem mã thẻ và biển số xe ( hoặc ảnh) có khớp không. Nếu khớp thì thực hiện thanh toán tiền, ngược lại báo lỗi.
Số lượng xe vào ra trong bãi biến động nhiều nên dùng danh sách liên kết để quản lý thông tin các xe đang có trong bãi sẽ tối ưu hơn là mảng).

## Tư duy logic và Thiết kế

Trước tiên, chúng ta cần định nghĩa các "vật thể" trong hệ thống.

### 1. "Vật thể" cơ bản: `VehicleNode` (Nút thông tin xe)

Đây chính là "mắt xích" (Node) trong danh sách liên kết. Mỗi `VehicleNode` sẽ đại diện cho **một chiếc xe đang gửi trong bãi**. Nó cần lưu trữ toàn bộ thông tin "lúc vào":

* `card_id`: Mã thẻ gửi xe (dùng để tìm kiếm).
* `license_plate`: Biển số xe (dùng để xác thực).
* `time_in`: Thời điểm xe vào (dùng để tính tiền).
* `photo_in`: Ảnh xe lúc vào (dùng để xác thực, nếu có).
* `next`: Con trỏ (tham chiếu) đến `VehicleNode` tiếp theo trong bãi.

### 2. "Vật thể" quản lý: `ParkingLot` (Bãi đỗ xe)

Đây là `class` sẽ quản lý toàn bộ danh sách liên kết. Nó chỉ cần một thuộc tính quan trọng nhất:

* `head`: Trỏ đến `VehicleNode` đầu tiên trong danh sách (chiếc xe đầu tiên). Nếu bãi rỗng, `head` là `None`.

### 3. Logic các thao tác chính

#### A. Gửi xe (`park_vehicle`)

Đây là thao tác **chèn (insert)** một `VehicleNode` mới vào danh sách.

1.  **Tiếp nhận thông tin:** Nhận `card_id`, `license_plate`, `time_in`, `photo_in`.
2.  **Kiểm tra trùng lặp:** *Nên làm*. Duyệt qua danh sách (O(n)) để xem `card_id` hoặc `license_plate` này đã tồn tại trong bãi chưa. Nếu có, báo lỗi.
3.  **Tạo Nút mới:** `new_vehicle = VehicleNode(...)`.
4.  **Chèn vào danh sách:** Cách nhanh nhất cho Linked List là **chèn vào đầu (prepend)**, thao tác này là O(1).
    * `new_vehicle.next = self.head` (Cho xe mới trỏ vào xe `head` cũ).
    * `self.head = new_vehicle` (Cập nhật `head` là xe mới).
5.  Thông báo gửi xe thành công.

#### B. Lấy xe (`exit_vehicle`)

Đây là thao tác phức tạp nhất: **Tìm kiếm (search)** + **Xác thực (validate)** + **Xóa (remove)**.

1.  **Tiếp nhận thông tin:** Nhận `card_id`, `license_plate_exit`, `time_out`, `photo_out`.
2.  **Tìm kiếm (O(n)):**
    * Đây là mấu chốt. Chúng ta cần duyệt (traverse) danh sách từ `self.head` để tìm `VehicleNode` có `card_id` khớp.
    * Chúng ta cần **2 con trỏ** để duyệt: `current` (nút đang xét) và `previous` (nút ngay trước `current`). Tại sao? Vì khi xóa `current`, chúng ta cần nối `previous` với `current.next`.
    * `current = self.head`, `previous = None`
    * `while current is not None:`
        * `if current.card_id == card_id_exit:` -> **Tìm thấy!** Dừng vòng lặp.
        * `else:` -> `previous = current`, `current = current.next` (Đi tiếp).
3.  **Xử lý kết quả tìm kiếm:**
    * **Trường hợp 1: Không tìm thấy (`current is None`)**
        * Vòng lặp chạy hết mà không tìm thấy.
        * Báo lỗi: "Lỗi: Mã thẻ không tồn tại trong bãi."
    * **Trường hợp 2: Tìm thấy (`current is not None`)**
        * Chuyển sang bước Xác thực.
4.  **Xác thực:**
    * So sánh biển số: `if current.license_plate != license_plate_exit:`
        * Báo lỗi: "Lỗi: Biển số không khớp với mã thẻ."
        * *(Ghi chú: Đề bài có nói "hoặc ảnh". So sánh ảnh là bài toán Computer Vision phức tạp. Trong code, chúng ta sẽ giả định chỉ so sánh biển số. Nếu muốn, có thể thêm một phép so sánh `current.photo_in != photo_out` nếu chúng chỉ là chuỗi).
5.  **Thanh toán và Xóa (Nếu xác thực thành công):**
    * **Tính tiền:**
        * Lấy `time_in = current.time_in`.
        * `duration = time_out - time_in` (Đây là đối tượng `timedelta`).
        * Viết một hàm `calculate_fee(duration)` (ví dụ: 5.000đ/giờ, làm tròn lên).
    * **Xóa Nút khỏi danh sách:**
        * *Nếu `previous is None`*: Xe cần xóa chính là `head`. Ta chỉ cần `self.head = current.next`.
        * *Nếu `previous is not None`*: Xe cần xóa ở giữa hoặc cuối. Ta "nối tắt": `previous.next = current.next`.
    * Thông báo thành công: In ra biển số, thời gian gửi, tổng tiền.

In [1]:
import datetime
import math

## --------------------------------------------------
## BƯỚC 1: ĐỊNH NGHĨA CÁC CLASS
## --------------------------------------------------

class VehicleNode:
    """
    Đại diện cho một "Nút" (Node) trong danh sách liên kết.
    Chính là thông tin của một xe đang gửi.
    """
    def __init__(self, card_id, license_plate, time_in, photo_in=None):
        self.card_id = card_id
        self.license_plate = license_plate
        self.time_in = time_in
        self.photo_in = photo_in
        self.next = None  # Con trỏ đến xe tiếp theo

    def __repr__(self):
        """Hàm hỗ trợ in thông tin Node cho dễ nhìn"""
        return f"[Card: {self.card_id}, Plate: {self.license_plate}, Time: {self.time_in}]"

class ParkingLot:
    """
    Đại diện cho "Bãi đỗ xe", quản lý danh sách liên kết.
    """
    def __init__(self, hourly_rate=5000):
        self.head = None  # Ban đầu bãi xe rỗng
        self.hourly_rate = hourly_rate
        self.count = 0

    def calculate_fee(self, time_in, time_out):
        """Tính phí gửi xe dựa trên thời gian."""
        duration = time_out - time_in
        seconds = duration.total_seconds()
        
        # Làm tròn lên giờ tiếp theo
        hours = math.ceil(seconds / 3600)
        
        # Đảm bảo tính phí ít nhất 1 giờ
        if hours == 0:
            hours = 1
            
        return hours * self.hourly_rate

    def find_vehicle_by_card(self, card_id):
        """
        Hàm nội bộ: Tìm một xe bằng mã thẻ.
        Trả về (previous_node, found_node)
        """
        current = self.head
        previous = None
        while current is not None:
            if current.card_id == card_id:
                return previous, current # Tìm thấy
            previous = current
            current = current.next
        
        return None, None # Không tìm thấy

    ## --------------------------------------------------
    ## LOGIC 1: GỬI XE (PARK VEHICLE)
    ## --------------------------------------------------
    def park_vehicle(self, card_id, license_plate, time_in, photo_in=None):
        """Thêm một xe mới vào bãi (chèn vào đầu danh sách)."""
        
        # 1. Kiểm tra trùng lặp (Tùy chọn nhưng nên có)
        _prev, found = self.find_vehicle_by_card(card_id)
        if found:
            print(f"LỖI: Mã thẻ '{card_id}' đã được sử dụng.")
            return

        # 2. Tạo Node xe mới
        new_vehicle = VehicleNode(card_id, license_plate, time_in, photo_in)
        
        # 3. Chèn vào đầu danh sách (O(1))
        new_vehicle.next = self.head
        self.head = new_vehicle
        self.count += 1
        
        print(f"\n[VÀO BÃI] Xe {license_plate} (Thẻ: {card_id}) đã vào bãi lúc {time_in}.")

    ## --------------------------------------------------
    ## LOGIC 2: LẤY XE (EXIT VEHICLE)
    ## --------------------------------------------------
    def exit_vehicle(self, card_id_exit, license_plate_exit, time_out, photo_out=None):
        """Tìm, xác thực, tính phí và xóa xe khỏi bãi."""
        
        print(f"\n[RA BÃI] Yêu cầu lấy xe: Thẻ {card_id_exit}, Biển số {license_plate_exit}...")
        
        # 1. Tìm xe bằng mã thẻ (O(n) search)
        previous, vehicle_to_exit = self.find_vehicle_by_card(card_id_exit)
        
        # 2. Xử lý kết quả tìm kiếm
        if vehicle_to_exit is None:
            print(f"LỖI: Không tìm thấy xe nào có mã thẻ '{card_id_exit}'.")
            return

        # 3. Xác thực thông tin
        # (Bỏ qua so sánh ảnh vì phức tạp, chỉ so sánh biển số)
        if vehicle_to_exit.license_plate != license_plate_exit:
            print(f"LỖI XÁC THỰC: Biển số '{license_plate_exit}' không khớp với '{vehicle_to_exit.license_plate}' của thẻ.")
            return

        # 4. Xác thực thành công -> Tính phí và Xóa
        print("Xác thực thành công!")
        
        # 4a. Tính phí
        fee = self.calculate_fee(vehicle_to_exit.time_in, time_out)
        
        # 4b. Xóa Node khỏi danh sách
        if previous is None:
            # Xe cần xóa là 'head'
            self.head = vehicle_to_exit.next
        else:
            # Xe cần xóa ở giữa hoặc cuối
            previous.next = vehicle_to_exit.next
            
        self.count -= 1
        
        # 5. In hóa đơn
        print("--- HÓA ĐƠN ---")
        print(f"Biển số xe: {vehicle_to_exit.license_plate}")
        print(f"Giờ vào:     {vehicle_to_exit.time_in}")
        print(f"Giờ ra:       {time_out}")
        print(f"Tổng thời gian: {time_out - vehicle_to_exit.time_in}")
        print(f"PHÍ PHẢI TRẢ: {fee} VND")
        print("------------------")

    def display_vehicles(self):
        """In tất cả các xe đang có trong bãi."""
        print(f"\n--- DANH SÁCH XE TRONG BÃI (Tổng: {self.count}) ---")
        if self.head is None:
            print("Bãi xe rỗng.")
            return
            
        current = self.head
        i = 1
        while current is not None:
            print(f"{i}. {current}")
            current = current.next
            i += 1
        print("------------------------------------------")

## --------------------------------------------------
## BƯỚC 3: CHẠY THỬ MÔ PHỎNG
## --------------------------------------------------

# Khởi tạo bãi xe
my_parking = ParkingLot(hourly_rate=3000) # 3000/giờ

# Dùng datetime để mô phỏng thời gian
now = datetime.datetime.now()

# 1. Xe 1 vào
my_parking.park_vehicle(
    card_id="T001", 
    license_plate="29-H1-12345", 
    time_in=now - datetime.timedelta(hours=4, minutes=30) # Vào cách đây 4.5 tiếng
)

# 2. Xe 2 vào
my_parking.park_vehicle(
    card_id="T002", 
    license_plate="30-K1-55555", 
    time_in=now - datetime.timedelta(hours=1, minutes=10) # Vào cách đây 1h 10p
)

# 3. Xe 3 vào
my_parking.park_vehicle(
    card_id="T003", 
    license_plate="99-B1-98765", 
    time_in=now - datetime.timedelta(minutes=20) # Vào cách đây 20p
)

# In danh sách xe hiện tại
my_parking.display_vehicles()

# Kịch bản 1: Lấy xe T002 thành công
# (Vào 1h 10p trước -> tính 2 giờ)
print("\n=== Kịch bản 1: Lấy xe T002 thành công ===")
my_parking.exit_vehicle(
    card_id_exit="T002",
    license_plate_exit="30-K1-55555",
    time_out=now
)

# Kịch bản 2: Lấy xe T001 nhưng sai biển số
print("\n=== Kịch bản 2: Lấy xe T001 sai biển số ===")
my_parking.exit_vehicle(
    card_id_exit="T001",
    license_plate_exit="29-H1-00000", # Sai biển số
    time_out=now
)

# Kịch bản 3: Lấy xe bằng mã thẻ không tồn tại
print("\n=== Kịch bản 3: Mã thẻ không tồn tại ===")
my_parking.exit_vehicle(
    card_id_exit="T999",
    license_plate_exit="12-A1-11111",
    time_out=now
)

# Kịch bản 4: Lấy xe T001 thành công
# (Vào 4h 30p trước -> tính 5 giờ)
print("\n=== Kịch bản 4: Lấy xe T001 thành công ===")
my_parking.exit_vehicle(
    card_id_exit="T001",
    license_plate_exit="29-H1-12345",
    time_out=now
)

# In danh sách xe cuối cùng (chỉ còn lại T003)
my_parking.display_vehicles()


[VÀO BÃI] Xe 29-H1-12345 (Thẻ: T001) đã vào bãi lúc 2025-11-10 04:42:23.590264.

[VÀO BÃI] Xe 30-K1-55555 (Thẻ: T002) đã vào bãi lúc 2025-11-10 08:02:23.590264.

[VÀO BÃI] Xe 99-B1-98765 (Thẻ: T003) đã vào bãi lúc 2025-11-10 08:52:23.590264.

--- DANH SÁCH XE TRONG BÃI (Tổng: 3) ---
1. [Card: T003, Plate: 99-B1-98765, Time: 2025-11-10 08:52:23.590264]
2. [Card: T002, Plate: 30-K1-55555, Time: 2025-11-10 08:02:23.590264]
3. [Card: T001, Plate: 29-H1-12345, Time: 2025-11-10 04:42:23.590264]
------------------------------------------

=== Kịch bản 1: Lấy xe T002 thành công ===

[RA BÃI] Yêu cầu lấy xe: Thẻ T002, Biển số 30-K1-55555...
Xác thực thành công!
--- HÓA ĐƠN ---
Biển số xe: 30-K1-55555
Giờ vào:     2025-11-10 08:02:23.590264
Giờ ra:       2025-11-10 09:12:23.590264
Tổng thời gian: 1:10:00
PHÍ PHẢI TRẢ: 6000 VND
------------------

=== Kịch bản 2: Lấy xe T001 sai biển số ===

[RA BÃI] Yêu cầu lấy xe: Thẻ T001, Biển số 29-H1-00000...
LỖI XÁC THỰC: Biển số '29-H1-00000' không khớp 